# Cinemática Inversa

## Base de dados

In [69]:
import os
from pandas import read_csv
from numpy import array
class save:
    def __init__(self,path_project):
        self.path_project = path_project
        
    def dataframe(self,data,path):
        path_data = os.path.join(self.path_project,path)
        data.to_csv(path_data,index=False)

class extract:
    def __init__(self,path_project):
        self.path_project = path_project

    def dataframe(self,path):
        path_data = os.path.join(self.path_project,path)
        return read_csv(path_data)
    
def partition(dataset,axis):
    return array(dataset.iloc[:,axis[0]:axis[1]])

In [81]:
import sys

path_module = [r'C:\Users\je7560\Downloads\matheus\kinematics-robotics\src\kinematicsrobotics',
               r'C:\Users\mathe\OneDrive\Graduação - UFC\Engenharia da Computação\TCC\Códigos e implementações\V.2\kinematics-robotics\src\kinematicsrobotics']
sys.path.append(path_module[1])

path_data = r'src\data\ready\dataset-radius-1.25cm.csv'
path_project = [r'C:\Users\mathe\OneDrive\Graduação - UFC\Engenharia da Computação\TCC\Códigos e implementações\V.2\kinematics-robotics',
                r'C:\Users\je7560\Downloads\matheus\kinematics-robotics']

ext = extract(path_project[0])

dataset = ext.dataframe(path_data)

y = partition(dataset,[0,4])
x = partition(dataset,[5,11])

### Análise da base de dados

In [71]:
print(f'Atributos e saídas:\n {dataset.columns}')
print(f"\nTamanho saída: {y.shape}\nTamanho entrada: {x.shape}\n")
print(f"Primeiras sáidas:\n{y[0:3]}\n")
print(f"Primeiras entrada:\n{x[0:3]}\n")


Atributos e saídas:
 Index(['theta_1', 'theta_2', 'theta_3', 'theta_4', 'theta_5', 'p_x', 'p_y',
       'p_z', 'roll', 'pich', 'yaw'],
      dtype='object')

Tamanho saída: (6889, 4)
Tamanho entrada: (6889, 6)

Primeiras sáidas:
[[ 0  0 12  0]
 [ 0 12 12  0]
 [ 0 12 24  0]]

Primeiras entrada:
[[ 5.32133136e+01 -2.47712141e-15  2.51517913e+00  3.14159265e+00
  -1.36135682e+00  2.75203954e-16]
 [ 5.36066568e+01 -1.78965176e-15  1.37424104e+01  2.37853137e+00
  -1.57079633e+00  7.63061284e-01]
 [ 5.28199704e+01 -3.12453009e-15  6.25758957e+00  3.14159265e+00
  -1.36135682e+00  2.37151833e-16]]



## Pré-processamento dos dados

In [91]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from pandas import DataFrame

def split(x, y, test_size):
    return train_test_split(x, y, test_size=test_size)

def zscore(dt_train,dt_test):
    scaler = StandardScaler()
    dt_train = scaler.fit_transform(dt_train)
    dt_test = scaler.transform(dt_test)
    
    return dt_train, dt_test, scaler

def statics_data_split(scaler):
    return DataFrame({'Média': scaler.mean_, 'Desvio Padrão': scaler.scale_})



In [92]:
size_train = 0.7
size_val = 0.2
size_test = 0.1

x_train, x_test, y_train, y_test = split(x,y,test_size=size_test)


x_train, x_test, scaler_x = zscore(x_train,x_test)

y_train, _, scaler_y = zscore(y_train,y_test)

#### info

In [93]:
statics_x = statics_data_split(scaler_x)
statics_x

,Média,Desvio Padrão
0,8.227073,20.169702
1,14.207571,22.235937
2,33.911014,16.585681
3,0.548008,1.624623
4,-0.509291,1.020346
5,-0.907337,1.709544


In [94]:
statics_y = statics_data_split(scaler_y)
statics_y

,Média,Desvio Padrão
0,60.790463,36.434209
1,82.802256,32.009458
2,60.393779,34.673457
3,36.855238,36.876788


#### Análises

In [95]:
print(f"treino: {size_train}\nTeste: {size_test}\nValidação: {size_val}")
print(f"Tamanho dos dados de treino: {len(x_train)}")
print(f"Tamanho dos dados de teste: {len(x_test)}")

treino: 0.7
Teste: 0.1
Validação: 0.2
Tamanho dos dados de treino: 5851
Tamanho dos dados de teste: 651


## Modelos

In [7]:
from sklearn.neural_network import MLPRegressor
def MLP():
    # Número máximo de épocas durante o treinamento.
    EPOCHS = 1000
    # Número máximo de épocas sem mudança
    EPOCHS_NOCHANGE = 5
    # tolerância para a otimização
    ERROR = 1e-4
    model = MLPRegressor(max_iter = EPOCHS,
                         tol = ERROR,
                         n_iter_no_change = EPOCHS_NOCHANGE,
                         early_stopping = True,
                         verbose=False,
                         random_state=42)
    return model

#### Informações

In [8]:
model = MLP()
model.get_params()

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': True,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 1000,
 'momentum': 0.9,
 'n_iter_no_change': 5,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': 42,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

## Estimação dos hiperparâmetros

In [83]:
from sklearn.model_selection import ShuffleSplit,RandomizedSearchCV,GridSearchCV
from itertools import product
from sklearn.metrics import mean_squared_error,r2_score
from pandas import DataFrame

# Calcula a porcentagem relativa dos dados de treino e validação
def size_split(size_train,size_val):
    size_val = (1 - size_train/(size_train+size_val))
    return size_val

# Espaço de busca dos neurônios
def space_hidden(min_neurons,max_neurons,layers,step):
    # Lista que armazena os neurônios
    neuron_space = []
    
    # Gera a lista de possíveis números de neurônios em cada camada
    possible_neurons = list(range(min_neurons, max_neurons + 1,step))
    
    # Crie todas as combinações possíveis
    for num_layers in layers:
        neuron_combinations = list(product(possible_neurons, repeat=num_layers))
        neuron_space.extend(neuron_combinations)

    return neuron_space

# Parâmetro da rede mlp
def parameter_mlp(min_neurons,max_neurons,layers,step,func_act): 
    # Dicionário que armazena os hiperparâmetros
    param_grid = {
        'hidden_layer_sizes': space_hidden(min_neurons, max_neurons,layers,step),
        'activation': func_act
    }

    return param_grid

# Random Search
def RandomizedSearch(x, y, model, param_grid, scoring,cv, n_iter):
    # Configura os parâmetros da técnica de otimização 
    random_search = RandomizedSearchCV(estimator=model, 
                                       param_distributions=param_grid, 
                                       scoring=scoring, 
                                       cv=cv, 
                                       n_iter=n_iter, 
                                       random_state=42, 
                                       return_train_score=True,
                                       verbose=0)
    
    # Treina os modelos
    random_search.fit(x, y)

    # DataFrame que armazena os resultado dos hiperparâmetros
    history = DataFrame(random_search.cv_results_)

    # Melhor hiperparâmetro
    best_estimator = random_search.best_estimator_

    return history,best_estimator

# Grid Search
def GridSearch(x, y, model, param_grid, scoring, cv):
    # Configura os parâmetros da técnica de otimização 
    gridshearch = GridSearchCV(estimator=model,
                               param_grid=param_grid,
                               cv=cv,
                               scoring=scoring,
                               verbose=0,
                               return_train_score=True)
    
    # Treina os modelos
    gridshearch.fit(x, y)
    
    # Armazena os resultado dos hiperparâmetros
    history = DataFrame(gridshearch.cv_results_)
    
    best_estimator = gridshearch.best_estimator_

    return history,best_estimator

# Validação cruzada hold out
def holdout(n_splits,test_size):
    return ShuffleSplit(n_splits=n_splits, 
                      test_size=test_size, 
                      random_state=42)



In [10]:
# Salvar as métricas 
sv = save(path_project[0])
path_data_save =  r'src\data\ready\history.csv'

# Dados de treino e validação
size_val = size_split(size_train,size_val)

# Espaço de busca do grid search
min_neurons= 50
max_neurons = 300
step = 10
layers = [2,3]
func_act =  ['relu', 'tanh']

param_grid = parameter_mlp(min_neurons = min_neurons,
                           max_neurons = max_neurons,
                           layers=layers,
                           step=step,
                           func_act=func_act)


# validação cruzada hold out
n_splits = 5

# Número de amostras selecionadas no espaço de busca
n_iter = 1000

# Técnica de validação
cv = holdout(n_splits,size_val)

# inicializando a rede
mlp = MLP()

# APlicação do grid search
history,best_model = RandomizedSearch(x=x_train, 
                           y=y_train, 
                           model=mlp, 
                           param_grid=param_grid,
                           scoring = 'neg_mean_squared_error', 
                           cv=cv,
                           n_iter = n_iter)


sv.dataframe(history,path_data_save)



### Infomações

In [22]:
print(f"Número de parametros de camada oculta: {len(param_grid['hidden_layer_sizes'])}")
print(f"Atributos do retorno da gridsearsh: {history.columns}")

Número de parametros de camada oculta: 204183
Atributos do retorno da gridsearsh: Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_hidden_layer_sizes', 'param_activation', 'params',
       'split0_test_score', 'split1_test_score', 'split2_test_score',
       'split3_test_score', 'split4_test_score', 'mean_test_score',
       'std_test_score', 'rank_test_score', 'split0_train_score',
       'split1_train_score', 'split2_train_score', 'split3_train_score',
       'split4_train_score', 'mean_train_score', 'std_train_score'],
      dtype='object')


In [26]:
best_model

MLPRegressor(early_stopping=True, hidden_layer_sizes=(220, 120, 210, 210),
             max_iter=1000, n_iter_no_change=5, random_state=42)

In [27]:
history[['mean_test_score','std_test_score','mean_train_score', 'std_train_score','params']]

,mean_test_score,std_test_score,mean_train_score,std_train_score,params
0,-0.005603,0.000387,-0.004849,0.000581,"{'hidden_layer_sizes': (220, 120, 210, 210), '..."


## Métricas de Avaliação

In [22]:
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

def predict_mse(y_predic,Y_test):    
    return mean_squared_error(Y_test, y_predic,multioutput='raw_values')



def curve_loss(model):
    plt.plot(model.loss_curve_)
    plt.title('Erro Quadratico')
    plt.xlabel('Épocas')
    plt.ylabel('MSE')
    plt.show()

## Treinamento, validação e teste do modelo selecionado

In [96]:
path_data = r'src\data\ready\history.csv'
history = ext.dataframe(path_data)

history_best = history[history['rank_test_score'] == 1]

params = eval(history_best.iloc[0]['params'])

In [97]:
# Treina o modelo com hiperparâmetros selecionados
mlp = MLP()

mlp.set_params(**params)

mlp.fit(x_train,y_train)

MLPRegressor(early_stopping=True, hidden_layer_sizes=(120, 220, 290),
             max_iter=1000, n_iter_no_change=5, random_state=42)

In [101]:
# Metrica de dados de treino
y_predic = mlp.predict(x_train)

predict_mse(scaler_y.inverse_transform(y_predic), scaler_y.inverse_transform(y_train))

array([ 2.28341302,  4.65147181, 13.61223064,  7.63514085])

In [100]:
# Metrica dos dados de teste
y_predic = mlp.predict(x_test)
predict_mse(scaler_y.inverse_transform(y_predic), y_test)

array([ 2.38640785,  5.00797486, 16.30891522,  8.68590815])